# Exploring dataset & Database creation

Dataset's source:
https://www.kaggle.com/datasets/sharthz23/mts-library

## Contents
* [<font size=4>Exploring datasets</font>](#cell_01)
* [<font size=4>Creating database tables</font>](#cell_02)
* [<font size=4>Data preparing</font>](#cell_03)
* [<font size=4>Loading</font>](#cell_04)

## Common functions

In [6]:
import pandas as pd
import numpy as np
import library_connections as lc
from psycopg2 import Error
import random
from datetime import timedelta
from datetime import date

## Exploring datasets<a id='cell_01'></a>

### Interactions

The file contains information about user-book interactions

In [7]:
df_interactions = pd.read_csv('Datasets/interactions.csv')
df_interactions.head()

,user_id,item_id,progress,rating,start_date
0,126706,14433,80,NaN,2018-01-01
1,127290,140952,58,NaN,2018-01-01
2,66991,198453,89,NaN,2018-01-01
3,46791,83486,23,5.0,2018-01-01
4,79313,188770,88,5.0,2018-01-01


In [8]:
# Interactions count
df_interactions.shape[0]

1533078

In [9]:
# Rating values
df_interactions['rating'].unique()

array([nan,  5.,  4.,  2.,  3.,  1.])

### User features

In [10]:
df_users = pd.read_csv('Datasets/users.csv')
df_users.head()

,user_id,age,sex
0,1,45_54,NaN
1,2,18_24,0.0
2,3,65_inf,0.0
3,4,18_24,0.0
4,5,35_44,0.0


In [11]:
# Users count
df_users.shape[0]

142888

In [12]:
# Sex values
df_users['sex'].unique()

array([nan,  0.,  1.])

### Book features

In [13]:
df_items = pd.read_csv('Datasets/items.csv')
df_items.head()

,id,title,genres,authors,year
0,128115,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,210979,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,95632,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,247906,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,294280,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [14]:
# Books count
df_items.shape[0]

59599

In [15]:
# Year values
df_items['year'].unique()

array(['1886', '1894', '1869', ..., '2004, 2007', '2014–2019',
       '1965,1966,1967,1968'], dtype=object)

## Creating database tables <a id="#cell_02"></a>

From the file: e-library_database.sql

In [16]:
# Read sql_file and execute queries
with open('e-library_database.sql', 'r') as sql_file:
    sql_queries = sql_file.read()

queries = sql_queries.split(';')

for query in queries:
    if query.strip():
        lc.execute_query(query)

## Data preparing <a id="#cell_03"></a>

### Deleting interactions without user's feature

In [17]:
df_interactions = df_interactions.merge(df_users, how='inner', on='user_id')
df_interactions = df_interactions.drop(['age', 'sex'], axis=1)

In [18]:
# Changing types
df_interactions['start_date'] = pd.to_datetime(df_interactions['start_date'])
df_interactions['rating'] = df_interactions['rating'].astype('float32')

### Registration date

For analysis, I will need the user's registration date. It is not in this dataset. Therefore, I will fill it with a random date in the range of 45 days before the first interaction with the library, or a random date if there was none.

In [19]:
# Found the first interaction
df_dates = df_interactions[['user_id', 'start_date']].groupby('user_id')['start_date'].min().reset_index()

# And merge to users
df_users = df_users.merge(df_dates, how='left', on='user_id')

In [20]:
# Looking for random date
def random_reg_date(start_date):

    if pd.isna(start_date):

        return date(2018, 1, 1) + timedelta(days=random.randint(0, 720))
    
    else:

        return start_date - timedelta(days=random.randint(0, 45))
    
df_users['reg_date'] = pd.to_datetime(df_users['start_date'].apply(random_reg_date))

df_users = df_users.drop('start_date', axis=1)

In [21]:
df_users.head()

,user_id,age,sex,reg_date
0,1,45_54,NaN,2018-01-19
1,2,18_24,0.0,2018-02-04
2,3,65_inf,0.0,2018-06-23
3,4,18_24,0.0,2017-12-30
4,5,35_44,0.0,2017-12-24


### Payments

To calculate metrics related to monetization, I will generate a dataset with payments. 

The date of the first payment is random between the date of registration and the first book, the last payment is the month of the last interaction. 

Subscription cost is 5 euros per month.

In [22]:
# Looking for start and end of interactions
df_dates = df_interactions[['user_id', 'start_date']].groupby('user_id').agg({'start_date': ['min', 'max']}).reset_index()
df_dates.columns = ['user_id', 'interaction_start_date', 'interaction_end_date']

In [23]:
# Add registration date from users
df_dates = df_dates.merge(df_users, how='left', on='user_id')
df_dates = df_dates.drop(['age', 'sex'], axis=1)

In [24]:
df_dates.head()

,user_id,interaction_start_date,interaction_end_date,reg_date
0,1,2018-02-26,2019-02-22,2018-01-19
1,2,2018-02-14,2019-10-11,2018-02-04
2,3,2018-07-02,2019-12-23,2018-06-23
3,4,2018-01-28,2019-09-13,2017-12-30
4,5,2018-02-07,2019-01-04,2017-12-24


In [25]:
# Function to generate a random date between two given dates
def random_date(start_date, end_date):
    return start_date + timedelta(days=np.random.randint(0, (end_date - start_date).days + 1))

# Creating a new dataframe for monthly subscription payments
payment_data = []

for _, row in df_dates.iterrows():
    user_id = row['user_id']
    registration_date = row['reg_date']
    interaction_start_date = row['interaction_start_date']
    interaction_end_date = row['interaction_end_date']

    # Generating the date of the first payment
    first_payment_date = random_date(registration_date, interaction_start_date)

    # Generating payment dates until the last interaction
    current_payment_date = first_payment_date
    while current_payment_date <= interaction_end_date:
        payment_data.append({'user_id': user_id, 'payment_date': current_payment_date, 'amount': 5})
        current_payment_date += pd.DateOffset(months=1)

# Creating a dataframe with payments
df_payments = pd.DataFrame(payment_data)

df_payments.head()


,user_id,payment_date,amount
0,1,2018-02-03,5
1,1,2018-03-03,5
2,1,2018-04-03,5
3,1,2018-05-03,5
4,1,2018-06-03,5


## Loading <a id="#cell_04"></a>

In [26]:
# Function to load data from a CSV file into a PostgreSQL table
def load_data_to_postgres(connection, df, table_name, batch_size=500):
    try:
        # Replace NaN values with None (NULL) in the DataFrame
        df = df.replace({np.nan: None})

        # Create a cursor to perform database operations
        with connection.cursor() as cursor:
            # Split the DataFrame into batches of size batch_size
            for i in range(0, len(df), batch_size):
                batch_df = df.iloc[i:i + batch_size]

                # Iterate through the batch DataFrame and insert rows into the table
                for _, row in batch_df.iterrows():
                    # Modify the column names in the INSERT query based on the DataFrame's column names
                    columns = ', '.join(df.columns)
                    values = ', '.join(['%s'] * len(df.columns))
                    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({values});"
                    cursor.execute(insert_query, tuple(row))

        connection.commit()
        print(f"Data loaded successfully into {table_name}.")
    except Error as e:
        print(f"Error loading data to {table_name}: {e}")

In [27]:
# Create a connection to the PostgreSQL database
connection = lc.create_connection()

if connection:

   # Load data from CSV files to the corresponding tables
   load_data_to_postgres(connection, df_users, 'users')
   load_data_to_postgres(connection, df_items, 'items')
   load_data_to_postgres(connection, df_interactions, 'interactions')
   load_data_to_postgres(connection, df_payments, 'payments')

   # Close the database connection
   connection.close()

Data loaded successfully into users.
Data loaded successfully into items.
Data loaded successfully into interactions.
Data loaded successfully into payments.
